# Bienestar en Bogotá - OPM
## Juan Diego Mayorga 
## MCPP
## 2022-1
-----

### Programa final 

Este script nace a partir de del código "Bienestar en Bogotá" el cual se encuentra dentro de las carpetas que se compartieron. Este código esta automatizado para que el usuario pueda hacer los querys que requiera de OSM utilizando el Overpass API. 

In [2]:
# Packages 

# OSM
#%pip install overpass
#%pip install OSMPythonTools]

import requests 
import overpass
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass

# Data 
import pandas as pd
import json
from datetime import datetime,timedelta
import time

# Text 
import re

# Graphs 
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Nos conectamos con la API
api = overpass.API(endpoint="https://overpass.myserver/interpreter")
api = overpass.API(timeout=600)

# Dejamos el link abierto para los request 
op_url = "https://overpass-api.de/api/interpreter" 

In [37]:
def get_data(type, ubicación, building):

    '''
Esta función recibe tres argumetos
1. El tipo de estructura que se busca (amenity, building, etc)
2. En que parte se desea buscar la estructura
3. Que estructura especificamente esta buscando (Colegios, hospitales, edificios del gobierno, etc)

Como resultado final el usuario obtendra una base de datos que contenga lo siguiente
1. Latitud 
2. Longitud
3. Tipo de amenity (Previendo que se harán multiples requests)
    '''

    temp = pd.DataFrame([])


    # En esta primera linea construimos el query. Este cambia dependiendo del tipo de request
    p1 = """[out:json][timeout:50];"""
    p2 = """area[name=""" + ubicación + """]->.search;"""
    p3 =  """node[""" + type + """=""" + building + """](area.search);out body;out skel qt;"""
        
    # Esta sigueinte línea conecta el query 
    op_query = p1 + p2 + p3
        
    # Acá nos conectamos con la API y le pasamos el query
    response = requests.get(op_url, params={"data": op_query})

    # La respuesta viene en un formato json y por lo tanto necesitamos transformarla
    x = response.json()
    data = pd.json_normalize(x["elements"])

    # Puede que algunas busquedas no tengan información, por lo tanto, se debe preeveer esto y avisarle al usuario
    if data.empty == True:
        print("La ubicación "+str(ubicación)+" no tiene datos disponibles para el "+str(type)+": "+ str(building))
           
    else:

        # Extraemos la info que nos interesa, en este caso la longitud, latitud y nombre del establecimiento
        final = data[["lat", "lon", "id","tags.name"]]

        # Cambiamos el nombre de las variables
        final.rename(columns = {'lat':'Latitud', "lon":"Longitud", "tags.name":"Nombre"}, inplace = True)   
        # Le pegamos el nombre del amenity para hacer filtros mas adelante
        final.insert(0,"Type", str(type))

        # Le pegamos el nombre del edificio
        final.insert(0,"Building", str(building))

        # Pegamos a la base de datos los datos de cada uno de los outputs de las localidades
        temp = temp.append(final)

        # Guardamos el archivo en un formato csv; OJO que este se guarda en su directorio principal, ajustar esta parte del código
        temp.to_csv("output.csv")

    # La API no permite hacer cierto numero de querys al tiempo, por tanto, toca poner a dormir al código para que no nos saque 
    # Este puede ir cambiando, dado que vamos a hacer multiples request, puede que arroje errores a la hora de calcularlo 
    time.sleep(20)

    return temp

In [10]:
# Probamos que el get data funcione
get_data("amenity", "Colombia", "hospital")

c:\Users\juand\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Building,Type,Latitud,Longitud,id,Nombre,Ciudad
0,hospital,amenity,4.629515,-74.135744,294169444,Clínica de Occidente,NaN
1,hospital,amenity,4.711816,-74.030771,313416844,Clínica El Bosque,NaN
2,hospital,amenity,4.856215,-74.030608,314114536,Clínica Universidad de La Sabana,Chía
3,hospital,amenity,4.743580,-74.084289,316153049,Cami Suba,NaN
4,hospital,amenity,5.006597,-74.470116,317564539,Hospital de Villeta,NaN
...,...,...,...,...,...,...,...
2313,hospital,amenity,11.778529,-72.444817,4393784622,NaN,NaN
2314,hospital,amenity,11.778615,-72.444821,4393784623,NaN,NaN
2315,hospital,amenity,12.174750,-71.281992,1182868504,NaN,NaN
2316,hospital,amenity,12.174880,-71.281931,6110198986,NaN,NaN


In [32]:
# Ahora podemos combinar el código con un formato initeractivo con el usuario

def user_input():

    '''
Esta función le pide al usuario la informacion que desee descargar de OSM.
Para esto se requiere proporcionar lo siguiente 

1. Grupo de edificaciones que desee descargar
2. Ubicación donde realizar la busqueda
3. Edificación específica que desea descargar

    '''

    print("\n¿Qué grupo de edifiaciones esta buscando? (Ejemplo: Amenity) >>")
    tipo = input()
    print("\n¿En que ubicación le gustaría buscarla? (Ejemplo: Bogotá D.C) >>")
    loc = input()
    print("\nTipo de edificación especifica que busca (Ejemplo: Hospital) >>")
    edi = input()

    return([tipo, loc, edi])

In [33]:
# Programa final 
def query_OSM():
    input = user_input()
    data = get_data(input[0], input[1], input[2])
    return data

In [35]:
# Esta es la función final que le permitirá al usuario buscar cualquier tipo de amenity 
query_OSM()


¿Qué grupo de edifiaciones esta buscando? (Ejemplo: Amenity) >>

¿En que ubicación le gustaría buscarla? (Ejemplo: Bogotá D.C) >>

Tipo de edificación especifica que busca (Ejemplo: Hospital) >>


c:\Users\juand\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Building,Type,Latitud,Longitud,id,Nombre
0,hospital,amenity,4.629515,-74.135744,294169444,Clínica de Occidente
1,hospital,amenity,4.711816,-74.030771,313416844,Clínica El Bosque
2,hospital,amenity,4.856215,-74.030608,314114536,Clínica Universidad de La Sabana
3,hospital,amenity,4.743580,-74.084289,316153049,Cami Suba
4,hospital,amenity,5.006597,-74.470116,317564539,Hospital de Villeta
...,...,...,...,...,...,...
2313,hospital,amenity,11.778529,-72.444817,4393784622,NaN
2314,hospital,amenity,11.778615,-72.444821,4393784623,NaN
2315,hospital,amenity,12.174750,-71.281992,1182868504,NaN
2316,hospital,amenity,12.174880,-71.281931,6110198986,NaN


In [40]:
country = ["Colombia", "Chile", "Argentina"]
for i in country:
    print(str(i))

Colombia
Chile
Argentina


In [44]:
# El usuario incluso podría hacer loop entre varias ubicaciones. Utilizando la función get data
country = ["Colombia", "Chile", "Argentina"]

data = pd.DataFrame([])

for i in country:
    x = get_data("amenity", str(i), "hospital")
    # para diferenciar los datos, el usuario puede agregar una columna que tenga el nombre de cada país
    x.insert(0,"country", str(i))
    data = data.append(x)

data


c:\Users\juand\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,country,Building,Type,Latitud,Longitud,id,Nombre
0,Colombia,hospital,amenity,4.629515,-74.135744,294169444,Clínica de Occidente
1,Colombia,hospital,amenity,4.711816,-74.030771,313416844,Clínica El Bosque
2,Colombia,hospital,amenity,4.856215,-74.030608,314114536,Clínica Universidad de La Sabana
3,Colombia,hospital,amenity,4.743580,-74.084289,316153049,Cami Suba
4,Colombia,hospital,amenity,5.006597,-74.470116,317564539,Hospital de Villeta
...,...,...,...,...,...,...,...
3039,Argentina,hospital,amenity,-23.611453,-62.594000,3372636585,NaN
3040,Argentina,hospital,amenity,-22.244130,-63.736195,5238803950,NaN
3041,Argentina,hospital,amenity,-22.740787,-62.497906,7404158668,NaN
3042,Argentina,hospital,amenity,-22.285782,-62.714965,4783269247,NaN
